In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
langchain_hub_key = os.getenv("LANGCHAIN_HUB_KEY")

In [2]:
from langchain_openai import ChatOpenAI
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain.prompts import PromptTemplate

In [3]:
llm = ChatOpenAI(openai_api_key=openai_api_key)

In [4]:
graph = Neo4jGraph(
    url="bolt://3.239.79.108:7687",
    username="neo4j",
    password="arrows-sewer-assistants",
)

In [5]:
CYPHER_GENERATION_TEMPLATE = """
You are an expert Neo4j Developer translating user questions into Cypher to answer questions about movies and provide recommendations.
Convert the user's question based on the schema.

Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
For movie titles that begin with "The", move "the" to the end, For example "The 39 Steps" becomes "39 Steps, The" or "The Matrix" becomes "Matrix, The".

If no data is returned, do not attempt to answer the question.
Only respond to questions that require you to construct a Cypher statement.
Do not include any explanations or apologies in your responses.

Examples: 

Find movies and genres:
MATCH (m:Movie)-[:IN_GENRE]->(g)
RETURN m.title, g.name

Schema: {schema}
Question: {question}
"""

In [6]:
cypher_generation_prompt = PromptTemplate(
    template=CYPHER_GENERATION_TEMPLATE,
    input_variables=["schema", "question"],
)

In [7]:
cypher_chain = GraphCypherQAChain.from_llm(
    llm,
    graph=graph,
    cypher_prompt=cypher_generation_prompt,
    verbose=True
)

In [8]:
cypher_chain.invoke({"query": "What movies has Tom Hanks directed and what are the genres?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (a:Actor {name: "Tom Hanks"})-[:DIRECTED]->(m:Movie)-[:IN_GENRE]->(g:Genre)
RETURN m.title, g.name
Full Context:
[{'m.title': 'That Thing You Do!', 'g.name': 'Drama'}, {'m.title': 'That Thing You Do!', 'g.name': 'Comedy'}, {'m.title': 'Larry Crowne', 'g.name': 'Drama'}, {'m.title': 'Larry Crowne', 'g.name': 'Romance'}, {'m.title': 'Larry Crowne', 'g.name': 'Comedy'}]

> Finished chain.


{'query': 'What movies has Tom Hanks directed and what are the genres?',
 'result': 'Tom Hanks has directed "That Thing You Do!" which falls under the Drama and Comedy genres, as well as "Larry Crowne" which is categorized as Drama, Romance, and Comedy.'}